# Garbage Classifier

Let's build a garbage classifier with machine learning.

## Data Model

We will start with downloading an existing data model.
For this we will use this one from kaggle [garbage-classification](https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification)

In [ ]:
import zipfile
path = Path('Garbage classification/Garbage classification')
if not path.exists():
    zipfile.ZipFile('garbage-classifier-dataset.zip').extractall()

In [ ]:
from fastai.data.transforms import get_image_files

fns = get_image_files(path)
fns

When we create our own dataset from images downloaded from the internet it's usually a good practice to verify that the images can be opened. In this example it's not really needed since we are working with a curated set.

In [ ]:
from fastai.vision.utils import verify_images

failed = verify_images(fns)

For machine learning to be able to use our dataset the set needs to be constructed in a consistent way. For this we can use a DataBlock which is a generic container used to quickly build 'Datasets' and 'DataLoaders'.

In [ ]:
from fastai.vision.augment import Resize
from fastai.data.transforms import RandomSplitter, parent_label
from fastai.vision.data import ImageBlock
from fastai.data.block import DataBlock, CategoryBlock

garbage = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

dls = garbage.dataloaders(path, verbose=True)

Let's see some of the files that we loaded.

In [ ]:
dls.show_batch(max_n=10, nrows=2)

## Traing our model
We are now ready to train our model.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(5)

Let's see how it did out of the box. One way is to build a Confusion Matrix

In [ ]:
from fastai.interpret import ClassificationInterpretation

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

We can also see where our model made its errors

In [ ]:
interp.plot_top_losses(5, nrows=1)

## Sharing our model
For now, though this will do. Let's export our trained model to a file.

In [ ]:
learn.export('garbage-classifier.pkl')